<a href="https://colab.research.google.com/github/suruchi2010/SSN-projects/blob/main/Dysarthic_speech_analysis_using_Neural_N_w.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import numpy as np
import pandas as pd
import statistics
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.mstats import zscore
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from glob import glob 
#glob()

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

path = "/content/gdrive/My Drive/UA_PHASE"
audio_files= glob(path + '/*/*/*.wav', recursive=True)
audio_files
#len(audio_files)

Mounted at /content/gdrive


['/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_12.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_23.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_18.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_24.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_22.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_15.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_13.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_16.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_14.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/digits/EIGHT_17.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/letters/ALPHA_15.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/letters/ALPHA_23.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/letters/ALPHA_16.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/letters/ALPHA_12.wav',
 '/content/gdrive/My Drive/UA_PHASE/F02_MILD/letters/ALPHA

In [ ]:
for file in (audio_files):
        #file_name=os.path.basename(file)
         type(file)
        # We create an empty list where we will append all the speakers severity for 
        # each row of our dataframe by slicing the file name 
         speaker_severity = []
         for i in audio_files:
           #print(i)
           x=i.split('/')[5]
           speaker_severity.append(x.split('_')[1])
           
#len(speaker_severity)
speaker_severity

['MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'MILD',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'HIGH',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW',
 'LOW'

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
        return result

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in audio_files:

        # We create an empty list where we will append all the speakers severity for 
        # each row of our dataframe by slicing the file name 
      speaker_severity = []
      for i in audio_files:
        p=i.split('/')[5]
        speaker_severity.append(p.split('_')[1])
      y=speaker_severity
      feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
      x.append(feature)
    return train_test_split(np.array(x), np.array(y), test_size=test_size, random_state=9)

In [ ]:
# Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
print((x_train.shape[0], x_test.shape[0]))

(172, 58)


In [ ]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
x_train

array([[-1.94079773e+02,  5.69517517e+01, -9.75878239e+00, ...,
         1.59881124e-03,  1.72159681e-03,  2.60861404e-03],
       [-2.78658325e+02,  8.08139267e+01, -8.64119434e+00, ...,
         2.66343181e-04,  4.71498206e-04,  3.75899748e-04],
       [-1.53271652e+02,  2.99966736e+01,  5.30330896e+00, ...,
         4.93320962e-03,  3.74405249e-03,  2.86579318e-03],
       ...,
       [-1.57972168e+02,  8.06335144e+01, -1.94173717e+01, ...,
         3.47955152e-03,  1.30253332e-03,  1.04480225e-03],
       [-2.26778503e+02,  7.44884033e+01,  1.27256355e+01, ...,
         9.29522561e-04,  9.05037043e-04,  8.05109972e-04],
       [-2.06695480e+02,  8.00073013e+01, -1.69285345e+00, ...,
         6.68197696e-04,  7.97775865e-04,  9.34533251e-04]])

In [ ]:
x_test

array([[-1.41099091e+02,  7.27034302e+01, -6.98947906e+00, ...,
         6.42731553e-03,  1.22401165e-02,  1.26556000e-02],
       [-1.55954269e+02,  8.41577988e+01,  3.83273277e+01, ...,
         2.71459877e-01,  1.20004006e-01,  4.04246598e-02],
       [-1.54223495e+02,  2.74929600e+01, -5.55585861e+00, ...,
         1.79887563e-03,  2.77071353e-03,  2.88704410e-03],
       ...,
       [-2.25702927e+02,  6.37568550e+01,  1.66283512e+01, ...,
         3.11195850e-03,  5.42705692e-03,  1.33705577e-02],
       [-1.80603592e+02,  5.24315872e+01,  3.73338848e-01, ...,
         1.79495290e-02,  1.24545870e-02,  8.84552207e-03],
       [-2.28668015e+02,  4.69089432e+01,  2.06254044e+01, ...,
         2.70805997e-03,  2.03375891e-03,  2.08570319e-03]])

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

In [ ]:
x_test

array([[ 1.52047046, -0.08905214, -0.30076871, ..., -0.11474093,
         0.56205561,  1.25580789],
       [ 1.16354443,  0.48714311,  1.83411282, ...,  9.03773158,
         9.88849851,  5.37460988],
       [ 1.20512981, -2.36329868, -0.23323063, ..., -0.27457665,
        -0.25747537, -0.19309774],
       ...,
       [-0.51230959, -0.53909651,  0.8118708 , ..., -0.22923144,
        -0.02758171,  1.36185287],
       [ 0.57129409, -1.10879753,  0.04609474, ...,  0.28316015,
         0.580617  ,  0.6906841 ],
       [-0.58355189, -1.38660606,  1.00017257, ..., -0.24317943,
        -0.32125523, -0.31195537]])

In [ ]:

# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred


array(['MODERATE', 'HIGH', 'HIGH', 'HIGH', 'MOD', 'HIGH', 'MOD', 'MOD',
       'MOD', 'MODERATE', 'LOW', 'HIGH', 'HIGH', 'MOD', 'LOW', 'MILD',
       'HIGH', 'MOD', 'MOD', 'MILD', 'HIGH', 'HIGH', 'HIGH', 'HIGH',
       'MILD', 'HIGH', 'HIGH', 'MILD', 'MILD', 'MODERATE', 'MILD',
       'MODERATE', 'MILD', 'MILD', 'LOW', 'MILD', 'MODERATE', 'MILD',
       'MILD', 'MILD', 'MILD', 'LOW', 'MILD', 'MODERATE', 'MODERATE',
       'MILD', 'LOW', 'HIGH', 'LOW', 'MILD', 'LOW', 'LOW', 'HIGH', 'HIGH',
       'HIGH', 'MILD', 'LOW', 'MILD'], dtype='<U8')

In [ ]:
#- Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#- Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 94.83%
